In [2]:
# -*- coding: utf-8 -*-
"""
Automatically generated by Colab.
"""
!pip install pandas numpy matplotlib python-docx scipy xlsxwriter xlrd openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from docx import Document
from docx.shared import Inches
from google.colab import files
from scipy.optimize import minimize
import zipfile
import os


uploaded = files.upload()

if not os.path.exists('plots'):
    os.makedirs('plots')
if not os.path.exists('excels'):
    os.makedirs('excels')


summary_data = []
model_params_data = []
rmse_values_data = []

def calculate_h_index(y_data):
    sorted_cites = np.sort(y_data)[::-1]
    h_index = np.max(np.where(sorted_cites >= np.arange(1, len(sorted_cites) + 1))[0]) + 1
    return h_index

def calculate_abc(M, N, h):
    a = (M * h**2) / (M * N - (M + N) * h)
    b = (M * N * (M - h) * (N - h)) * (h / (M * N - (M + N) * h))**2
    c = (N * h**2) / (M * N - (M + N) * h)
    return a, b, c

def calculate_rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def power_law_new(x_data, M, N):
    C = M
    lam = np.log(M) / np.log(N)
    return C / (x_data ** lam)

def zip_dir(dir_path, ziph):
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            ziph.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(dir_path, '..')))


for file_name in uploaded.keys():
    xls = pd.ExcelFile(file_name)

    doc = Document()
    doc.add_heading('Plots for Each Sheet', 0)

    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name)
        x_data = df['Serial Number'].values
        y_data = df['Cites'].values

        M = np.max(y_data)
        N = len(y_data)
        h = calculate_h_index(y_data)

        a, b, c = calculate_abc(M, N, h)
        fitted_values_ce00h = (b / (x_data + c)) - a
        rmse_ce00h = calculate_rmse(y_data, fitted_values_ce00h)

        fitted_values_power_law_new = power_law_new(x_data, M, N)
        rmse_power_law_new = calculate_rmse(y_data, fitted_values_power_law_new)

        summary_data.append({
            'File': file_name,
            'Sheet': sheet_name,
            'M': M,
            'N': N,
            'h': h
        })

        model_params_data.append({
            'File': file_name,
            'Sheet': sheet_name,
            'Model': 'PL11',
            'Parameters': f"C: {M:.2f}, Lambda: {np.log(M) / np.log(N):.2f}, RMSE: {rmse_power_law_new:.2f}"
        })

        rmse_values_data.append({
            'File': file_name,
            'Sheet': sheet_name,
            'CE00h RMSE': rmse_ce00h,
            'PL11 RMSE': rmse_power_law_new
        })

        plt.figure(figsize=(10, 6))
        plt.scatter(x_data, y_data, label='Data Points')
        plt.plot(x_data, fitted_values_ce00h, label='CE00h')
        plt.plot(x_data, fitted_values_power_law_new, label='PL11')
        plt.title(f'Fitted Models for {sheet_name}')
        plt.xlabel('Serial Number')
        plt.ylabel('Cites')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()

        plt.savefig(f'plots/{file_name}_{sheet_name}.png')
        plt.close()

        doc.add_heading(f'Plot for {sheet_name}', level=1)
        doc.add_picture(f'plots/{file_name}_{sheet_name}.png', width=Inches(6))

    doc.save(f'excels/{file_name}_plots.docx')


summary_df = pd.DataFrame(summary_data)
model_params_df = pd.DataFrame(model_params_data)
rmse_values_df = pd.DataFrame(rmse_values_data)

summary_df.to_excel('summary.xlsx', index=False)
model_params_df.to_excel('model_params.xlsx', index=False)
rmse_values_df.to_excel('rmse_values.xlsx', index=False)


with zipfile.ZipFile('output.zip', 'w') as zipf:
    zipf.write('summary.xlsx')
    zipf.write('model_params.xlsx')
    zipf.write('rmse_values.xlsx')
    zip_dir('plots', zipf)
    zip_dir('excels', zipf)


files.download('output.zip')


Saving Biz Processed.xlsx to Biz Processed.xlsx
Saving processed_Arts Data.xlsx to processed_Arts Data.xlsx
Saving processed_Med Data.xlsx to processed_Med Data.xlsx
Saving processed_Phy and math Data.xlsx to processed_Phy and math Data.xlsx
Saving processed_Soc Data.xlsx to processed_Soc Data.xlsx
Saving processed_Chem Data.xlsx to processed_Chem Data.xlsx
Saving processed_ECS Data.xlsx to processed_ECS Data.xlsx
Saving processed_Life Data.xlsx to processed_Life Data.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>